In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import xgboost as xgb
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [8]:
df = pd.read_csv('../data/train_data_encoded_full_1.csv')
df.price = np.log(df.price)
print('shape:', df.shape)
df.head(5)

shape: (36944, 8)


,quality,color,clarity,price,weight_ES,depth_percent_ES,table_percent_ES,volume_ES
0,3,6,1,1.848927,-1.093613,0.556467,0.364016,-1.094400
1,4,5,5,2.217354,0.679178,0.827088,-0.626742,0.730946
2,4,4,3,2.077314,-0.044920,0.015227,0.859395,-0.065353
3,2,3,1,2.124773,0.853961,1.278121,-0.131363,0.848234
4,3,3,4,1.885250,-0.943800,0.466260,0.859395,-0.966099


In [9]:
#split dara train and test
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

#print("Datos de entrenamiento")
#print("-----------------------")
#print(y_train.describe())
#print('\n')
#print("Datos de testeo")
#print("-----------------------")
#print(y_test.describe())

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [11]:
params = { 'max_depth': [7],
           'learning_rate': [0.02],
           'n_estimators': [750],
           'colsample_bytree': [0.9, 1]}
xgbr = xgb.XGBRegressor(seed = 20, objective ='reg:squarederror')
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=3,
                   n_jobs=-1)
clf.fit(X_train, y_train)

print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

y_pred_gb_test = clf.predict(X_test)
y_pred_gb_train= clf.predict(X_train)

results_xg_boost = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "XG Boost GridSearch")
results_xg_boost.style.background_gradient(cmap='coolwarm')

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 4/5] END colsample_bytree=0.9, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  16.4s
[CV 2/5] END colsample_bytree=0.9, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  17.1s
[CV 1/5] END colsample_bytree=0.9, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  17.2s
[CV 5/5] END colsample_bytree=0.9, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  17.1s
[CV 3/5] END colsample_bytree=0.9, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  17.2s
[CV 2/5] END colsample_bytree=1, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  18.0s
[CV 1/5] END colsample_bytree=1, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  18.1s
[CV 3/5] END colsample_bytree=1, learning_rate=0.02, max_depth=7, n_estimators=750;, score=-0.000 total time=  

,MAE,MSE,RMSE,R2,set,modelo
0,0.009495,0.000162,0.012739,0.990109,test,XG Boost GridSearch
1,0.008560,0.000131,0.011434,0.991925,train,XG Boost GridSearch


In [13]:
params = { 'max_depth': [5, 6, 7, 8],
           'learning_rate': [0.01, 0.02, 0.03],
           'n_estimators': [650, 750, 850],
           'colsample_bytree': [0.75, 0.85, 0.95]}
xgbr = xgb.XGBRegressor(seed = 20, objective ='reg:squarederror')
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=3,
                   n_jobs=-1)
clf.fit(X_train, y_train)

print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

y_pred_gb_test = clf.predict(X_test)
y_pred_gb_train= clf.predict(X_train)

results_xg_boost1 = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "XG Boost GridSearch 1")
results_xg_boost1.style.background_gradient(cmap='coolwarm')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 3/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=650;, score=-0.000 total time=   8.9s
[CV 2/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=650;, score=-0.000 total time=   9.0s
[CV 1/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=650;, score=-0.000 total time=   9.2s
[CV 5/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=650;, score=-0.000 total time=   9.3s
[CV 4/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=650;, score=-0.000 total time=   9.4s
[CV 1/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=750;, score=-0.000 total time=  10.7s
[CV 2/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=750;, score=-0.000 total time=  10.7s
[CV 3/5] END colsample_bytree=0.75, learning_rate=0.01, max_depth=5, n_estimators=750;, score=-0.

,MAE,MSE,RMSE,R2,set,modelo
0,0.009495,0.000162,0.012739,0.990109,test,XG Boost GridSearch 1
1,0.008560,0.000131,0.011434,0.991925,train,XG Boost GridSearch 1
